In [6]:
base_url = "http://43.200.7.56:8008/v1"
chat_model = "glm-4-flash"

In [7]:
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('ZISHU_API_KEY')
base_url = "http://43.200.7.56:8008/v1"
chat_model = "glm-4-flash"



from openai import OpenAI
client = OpenAI(
    api_key = api_key,
    base_url = base_url
)

## **结构化 prompt 的几个概念：**

**标识符**：#, <> 等符号(-, []也是)，这两个符号依次标识标题,变量，控制内容层级，用于标识层次结构。

**属性词**：Role, Profile, Initialization 等等，属性词包含语义，是对模块下内容的总结和提示，用于标识语义结构。

使用分隔符清晰标示输入的不同部分,像三重引号、XML标记、节标题等分隔符可以帮助标示需要以不同方式处理的文本部分。

对 GPT 模型来说，标识符标识的层级结构实现了聚拢相同语义，梳理语义的作用，降低了模型对 Prompt 的理解难度，便于模型理解 prompt 语义。

属性词实现了对 prompt 内容的语义提示和归纳作用，缓解了 Prompt 中不当内容的干扰。 使用属性词与 prompt 内容相结合，实现了局部的总分结构，便于模型提纲挈领的获得 prompt 整体语义。

一个好的结构化 Prompt 模板，某种意义上是构建了一个好的全局思维链。 如 LangGPT 中展示的模板设计时就考虑了如下思维链:

Role (角色) -> Profile（角色简介）—> Profile 下的 skill (角色技能) -> Rules (角色要遵守的规则) -> Workflow (满足上述条件的角色的工作流程) -> Initialization (进行正式开始工作的初始化准备) -> 开始实际使用

构建 Prompt 时，不妨参考优质模板的全局思维链路，熟练掌握后，完全可以对其进行增删改留调整得到一个适合自己使用的模板。例如当你需要控制输出格式，尤其是需要格式化输出时，完全可以增加 Ouput 或者 OutputFormat 这样的模块

## **prompt设计方法论**

1. 数据准备。收集高质量的案例数据作为后续分析的基础。
2. 模型选择。根据具体创作目的,选择合适的大语言模型。
3. 提示词设计。结合案例数据,设计初版提示词;注意角色设置、背景描述、目标定义、约束条件等要点。
4. 测试与迭代。将提示词输入 GPT 进行测试,分析结果;通过追问、深度交流、指出问题等方式与 GPT 进行交流,获取优化建议。
5. 修正提示词。根据 GPT 提供的反馈,调整提示词的各个部分,强化有效因素,消除无效因素。
6. 重复测试。输入经修正的提示词重新测试,比较结果,继续追问GPT并调整提示词。
7. 循环迭代。重复上述测试-交流-修正过程,直到结果满意为止。
8. 总结提炼。归纳提示词优化过程中获得的宝贵经验,形成设计提示词的最佳实践。
9. 应用拓展。将掌握的方法论应用到其他创意内容的设计中,不断丰富提示词设计的技能。

In [8]:

sys_prompt = """你是一个聪明的客服。您将能够根据用户的问题将不同的任务分配给不同的人。您有以下业务线：
1.用户注册。如果用户想要执行这样的操作，您应该发送一个带有"registered workers"的特殊令牌。并告诉用户您正在调用它。
2.用户数据查询。如果用户想要执行这样的操作，您应该发送一个带有"query workers"的特殊令牌。并告诉用户您正在调用它。
3.删除用户数据。如果用户想执行这种类型的操作，您应该发送一个带有"delete workers"的特殊令牌。并告诉用户您正在调用它。
"""
registered_prompt = """
您的任务是根据用户信息存储数据。您需要从用户那里获得以下信息：
1.用户名、性别、年龄
2.用户设置的密码
3.用户的电子邮件地址
如果用户没有提供此信息，您需要提示用户提供。如果用户提供了此信息，则需要将此信息存储在数据库中，并告诉用户注册成功。
存储方法是使用SQL语句。您可以使用SQL编写插入语句，并且需要生成用户ID并将其返回给用户。
如果用户没有新问题，您应该回复带有 "customer service" 的特殊令牌，以结束任务。
"""
query_prompt = """
您的任务是查询用户信息。您需要从用户那里获得以下信息：
1.用户ID
2.用户设置的密码
如果用户没有提供此信息，则需要提示用户提供。如果用户提供了此信息，那么需要查询数据库。如果用户ID和密码匹配，则需要返回用户的信息。
如果用户没有新问题，您应该回复带有 "customer service" 的特殊令牌，以结束任务。
"""
delete_prompt = """
您的任务是删除用户信息。您需要从用户那里获得以下信息：
1.用户ID
2.用户设置的密码
3.用户的电子邮件地址
如果用户没有提供此信息，则需要提示用户提供该信息。
如果用户提供了这些信息，则需要查询数据库。如果用户ID和密码匹配，您需要通知用户验证码已发送到他们的电子邮件，需要进行验证。
如果用户没有新问题，您应该回复带有 "customer service" 的特殊令牌，以结束任务。
"""

In [9]:
#智能客服
class SmartAssistant:
    def __init__(self):
        self.client = client 

        self.system_prompt = sys_prompt
        self.registered_prompt = registered_prompt
        self.query_prompt = query_prompt
        self.delete_prompt = delete_prompt

        # Using a dictionary to store different sets of messages
        self.messages = {
            "system": [{"role": "system", "content": self.system_prompt}],
            "registered": [{"role": "system", "content": self.registered_prompt}],
            "query": [{"role": "system", "content": self.query_prompt}],
            "delete": [{"role": "system", "content": self.delete_prompt}]
        }

        # Current assignment for handling messages
        self.current_assignment = "system"

    def get_response(self, user_input):
        self.messages[self.current_assignment].append({"role": "user", "content": user_input})
        while True:
            response = self.client.chat.completions.create(
                model=chat_model,
                messages=self.messages[self.current_assignment],
                temperature=0.9,
                stream=False,
                max_tokens=2000,
            )

            ai_response = response.choices[0].message.content
            if "registered workers" in ai_response:
                self.current_assignment = "registered"
                print("意图识别:",ai_response)
                print("switch to <registered>")
                self.messages[self.current_assignment].append({"role": "user", "content": user_input})
            elif "query workers" in ai_response:
                self.current_assignment = "query"
                print("意图识别:",ai_response)
                print("switch to <query>")
                self.messages[self.current_assignment].append({"role": "user", "content": user_input})
            elif "delete workers" in ai_response:
                self.current_assignment = "delete"
                print("意图识别:",ai_response)
                print("switch to <delete>")
                self.messages[self.current_assignment].append({"role": "user", "content": user_input})
            elif "customer service" in ai_response:
                print("意图识别:",ai_response)
                print("switch to <customer service>")
                self.messages["system"] += self.messages[self.current_assignment]
                self.current_assignment = "system"
                return ai_response
            else:
                self.messages[self.current_assignment].append({"role": "assistant", "content": ai_response})
                return ai_response

    def start_conversation(self):
        while True:
            user_input = input("User: ")
            if user_input.lower() in ['exit', 'quit']:
                print("Exiting conversation.")
                break
            response = self.get_response(user_input)
            print("Assistant:", response)

In [10]:
assistant = SmartAssistant()
assistant.start_conversation()

# 运行助手
# if __name__ == "__main__":
#     assistant = SmartAssistant()
#     assistant.start_conversation()


意图识别: 好的，根据您提供的信息，我将执行用户数据查询的操作。

我已经发送了带有"query workers"的特殊令牌。接下来，我将调用数据查询工作人员来处理您提供的信息。

工作人员正在查询账号1001的相关数据，包括余额￥505和注册时间2021-10-01。请您稍等片刻，我们很快会为您提供更详细的信息。
switch to <query>
意图识别: 您提供的用户ID是1001，并声称您的余额是￥505，注册时间是2021-10-01。为了查询数据库以验证这些信息，我需要您提供用户设置的密码。请提供您的密码以便我进行查询。

如果您没有提供密码，或者您完成了查询，并且没有新问题，请回复带有 "customer service" 的特殊令牌。
switch to <customer service>
Assistant: 您提供的用户ID是1001，并声称您的余额是￥505，注册时间是2021-10-01。为了查询数据库以验证这些信息，我需要您提供用户设置的密码。请提供您的密码以便我进行查询。

如果您没有提供密码，或者您完成了查询，并且没有新问题，请回复带有 "customer service" 的特殊令牌。
意图识别: 好的，您正在查询用户数据。我将调用查询工作人员来处理您的请求。

[发送 "query workers" 令牌]

请稍等，正在查询您的账户信息。
switch to <query>
Assistant: 很抱歉，但我无法直接查询数据库或验证您的账户信息。如果您希望查询账户信息，请确保您提供的用户ID和密码是正确的，并且这些信息已经注册在我们的系统中。如果您提供了正确的用户ID和密码，请告诉我，我将模拟查询并返回用户信息。

请提供您的用户ID和密码，以便我继续。如果您不确定密码，您可能需要重置它。
意图识别: 很抱歉，但我无法进行实际查询。为了模拟这个过程，我将根据您提供的信息生成一些假定的用户信息。

根据您提供的用户ID（1001），以下是假定的用户信息：

用户信息：
- 用户ID：1001
- 余额：￥505
- 注册时间：2021-10-01

请注意，这些信息是基于您的输入生成的，并不代表真实的账户信息。

如果您需要进一步的帮助或有其他问题，请告诉我。如果您没有新问题，请回复带有 "customer service" 的

In [ ]:
import json
import re


def extract_json_content(text):
    # 这个函数的目标是提取大模型输出内容中的json部分，并对json中的换行符、首位空白符进行删除
    text = text.replace("\n","")
    pattern = r"```json(.*?)```"
    matches = re.findall(pattern, text, re.DOTALL)
    if matches:
        return matches[0].strip()
    return text

class JsonOutputParser:
    def parse(self, result):
        # 这个函数的目标是把json字符串解析成python对象
        # 其实这里写的这个函数性能很差，经常解析失败，有很大的优化空间
        try:
            result = extract_json_content(result)
            parsed_result = json.loads(result)
            return parsed_result
        except json.JSONDecodeError as e:
            raise Exception(f"Invalid json output: {result}") from e

class GradingOpenAI:
    def __init__(self):
        self.model = "glm-4-flash"
        self.output_parser = JsonOutputParser()
        self.template = """你是一位中国专利代理师考试阅卷专家，
擅长根据给定的题目和答案为考生生成符合要求的评分和中文评语，
并按照特定的格式输出。
你的任务是，根据我输入的考题和答案，针对考生的作答生成评分和中文的评语，并以JSON格式返回。
阅卷标准适当宽松一些，只要考生回答出基本的意思就应当给分。
答案如果有数字标注，含义是考生如果答出这个知识点，这道题就会得到几分。
生成的中文评语需要能够被json.loads()这个函数正确解析。
生成的整个中文评语需要用英文的双引号包裹，在被包裹的字符串内部，请用中文的双引号。
中文评语中不可以出现换行符、转义字符等等。

输出格式为JSON:
{{
  "llmgetscore": 0,
  "llmcomments": "中文评语"
}}

比较学生的回答与正确答案，
并给出满分为10分的评分和中文评语。 
题目：{ques_title} 
答案：{answer} 
学生的回复：{reply}"""

    def create_prompt(self, ques_title, answer, reply):
        return self.template.format(
            ques_title=ques_title,
            answer=answer,
            reply=reply
        )

    def grade_answer(self, ques_title, answer, reply):
        success = False
        while not success:
            # 这里是一个不得已的权宜之计
            # 上面的json解析函数不是表现很差吗，那就多生成几遍，直到解析成功
            # 对大模型生成的内容先解析一下，如果解析失败，就再让大模型生成一遍
            try:
                response = client.chat.completions.create(
                    model=self.model,
                    messages=[
                        {"role": "system", "content": "你是一位专业的考试阅卷专家。"},
                        {"role": "user", "content": self.create_prompt(ques_title, answer, reply)}
                    ],
                    temperature=0.7
                )

                result = self.output_parser.parse(response.choices[0].message.content)
                success = True
            except Exception as e:
                print(f"Error occurred: {e}")
                continue

        return result['llmgetscore'], result['llmcomments']

    def run(self, input_data):
        output = []
        for item in input_data:
            score, comment = self.grade_answer(
                item['ques_title'], 
                item['answer'], 
                item['reply']
            )
            item['llmgetscore'] = score
            item['llmcomments'] = comment
            output.append(item)
        return output
grading_openai = GradingOpenAI()

In [ ]:
# 示例输入数据
input_data = [
 {'ques_title': '请解释共有技术特征、区别技术特征、附加技术特征、必要技术特征的含义',
  'answer': '共有技术特征：与最接近的现有技术共有的技术特征（2.5分）； 区别技术特征：区别于最接近的现有技术的技术特征（2.5分）； 附加技术特征：对所引用的技术特征进一步限定的技术特征，增加的技术特征（2.5分）； 必要技术特征：为解决其技术问题所不可缺少的技术特征（2.5分）。',
  'fullscore': 10,
  'reply': '共有技术特征：与所对比的技术方案相同的技术特征\n区别技术特征：与所对比的技术方案相区别的技术特征\n附加技术特征：对引用的技术特征进一步限定的技术特征\n必要技术特征：解决技术问题必须可少的技术特征'},
 {'ques_title': '请解释前序部分、特征部分、引用部分、限定部分',
  'answer': '前序部分：独权中，主题+与最接近的现有技术共有的技术特征，在其特征在于之前（2.5分）； 特征部分：独权中，与区别于最接近的现有技术的技术特征，在其特征在于之后（2.5分）；引用部分：从权中引用的权利要求编号及主题 （2.5分）；限定部分：从权中附加技术特征（2.5分）。',
  'fullscore': 10,
  'reply': '前序部分：独立权利要求中与现有技术相同的技术特征\n特征部分：独立权利要求中区别于现有技术的技术特征\n引用部分：从属权利要求中引用其他权利要求的部分\n限定部分：对所引用的权利要求进一步限定的技术特征'}]

In [ ]:
# 运行Chain
graded_data = grading_openai.run(input_data)
print(graded_data)

In [1]:
pip install openai python-dotenv pydantic llama-index-embeddings-openai -i  https://pypi.tuna.tsinghua.edu.cn/simple 

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/6a/3e/b68c118422ec867fa7ab88444e1274aa40681c606d59ac27de5a5588f082/python_dotenv-1.0.1-py3-none-any.whl (19 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl (20 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/e9/44/75a9c9421471a6c4805dbf2356f7c181a29c1879239abab1ea2cc8f38b40/sniffio-1.3.1-py3-none-any.whl (10 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/78/b6/6307fbef88d9b5ee7421e68d78a9f162e0da4900bc5f5793f6d3d0e34fb8/annotated_types-0.7.0-py3-none-any.whl (13 kB)
     ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
     ------------------------------------ --- 1.8/2.0 MB 10.1 MB/s eta 0:00:01
     ---------------------------------------- 2.0/2.0 MB 8.5 MB/s eta 0:00:00
     -------------------------------------

In [2]:
pip  install llama-index-core llama-index-readers-file llama-index-vector-stores-faiss  -i  https://pypi.tuna.tsinghua.edu.cn/simple


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/b1/fe/e8c672695b37eecc5cbf43e1d0638d88d66ba3a44c4d321c796f4e59167f/beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
     ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
     ----- ---------------------------------- 1.6/11.5 MB 7.6 MB/s eta 0:00:02
     ---------- ----------------------------- 2.9/11.5 MB 7.3 MB/s eta 0:00:02
     ----------------- ---------------------- 5.0/11.5 MB 8.0 MB/s eta 0:00:01
     ------------------------ --------------- 7.1/11.5 MB 8.4 MB/s eta 0:00:01
     ------------------------------ --------- 8.9/11.5 MB 8.5 MB/s eta 0:00:01
     -------------------------------------- - 11.0/11.5 MB 8.8 MB/s eta 0:00:01
     ---------------------------------------- 11.5/11.5 MB 8.6 MB/s eta 0:00:00
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/d1/c2/fe97d779f3ef3b15f05c94a2f1e3d21732574ed441687474db9d342a7315/soupsieve-2.6-

In [ ]:
pip install llama-index-vector-stores-qdrant llamaindex-py-client qdrant-client requests  -i  https://pypi.tuna.tsinghua.edu.cn/simple 

In [3]:
pip install sqlalchemy pyodbc pymssql llama-index llama_index  -i  https://pypi.tuna.tsinghua.edu.cn/simple 

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
     -------------------------- ------------- 1.3/2.0 MB 5.1 MB/s eta 0:00:01
     ---------------------------------------- 2.0/2.0 MB 5.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install faiss -i  https://pypi.tuna.tsinghua.edu.cn/simple 

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss


In [5]:
pip install xinference xinference_client llama_index.embeddings.langchain  llama_index.embeddings.xinference -i  https://pypi.tuna.tsinghua.edu.cn/simple 

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/26.3 MB ? eta -:--:--
     ---------------------------------------- 0.3/26.3 MB ? eta -:--:--
     - -------------------------------------- 0.8/26.3 MB 3.4 MB/s eta 0:00:08
     --- ------------------------------------ 2.1/26.3 MB 4.5 MB/s eta 0:00:06
     ---- ----------------------------------- 2.9/26.3 MB 4.8 MB/s eta 0:00:05
     ------ --------------------------------- 4.2/26.3 MB 5.1 MB/s eta 0:00:05
     --------- ------------------------------ 6.0/26.3 MB 5.6 MB/s eta 0:00:04
     ----------- ---------------------------- 7.6/26.3 MB 5.9 MB/s eta 0:00:04
     -------------- ------------------------- 9.7/26.3 MB 6.5 MB/s eta 0:00:03
     ---------------- ----------------------- 11.0/26.3 MB 6.6 MB/s eta 0:00:03
     ------------------ --------------------- 12.1/26.3 MB 6.3 MB/s eta 0:00:03
     ------------------- -------------------- 12.6/26.3 MB 6.0 MB/s eta 0:00:03
  

In [11]:
pip install duckduckgo_search -i  https://pypi.tuna.tsinghua.edu.cn/simple 

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
     ------------- -------------------------- 1.0/3.2 MB 7.2 MB/s eta 0:00:01
     -------------------------------- ------- 2.6/3.2 MB 7.6 MB/s eta 0:00:01
     ---------------------------------------- 3.2/3.2 MB 7.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
     --------------------- ------------------ 2.1/3.8 MB 9.8 MB/s eta 0:00:01
     -------------------------------- ------- 3.1/3.8 MB 7.4 MB/s eta 0:00:01
     ---------------------------------------- 3.8/3.8 MB 7.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
